<a href="https://colab.research.google.com/github/pavel7479/AvitoTech/blob/master/scan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Установка библиотек

In [1]:
!pip install lxml
!pip install requests
!pip install beautifulsoup4
!pip install xlsxWriter
!pip install gspread
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.2 MB/s eta 0:00:00


In [2]:
import requests
import lxml
from bs4 import BeautifulSoup
import xlsxwriter
import pandas as pd
from openpyxl import load_workbook

# Обработка страницы с игрой

In [3]:
# Страница с игрой

url = '''
https://nb-bet.com/hockey/Events/199343-spartak-moskva-traktor-prognoz-na-match
'''
response = requests.get(url)
response.text

'<!DOCTYPE html><html lang="ru"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width,initial-scale=1,maximum-scale=1,user-scalable=no,minimal-ui"/><meta name="google" content="notranslate"/><link rel="apple-touch-icon" sizes="180x180" href="/img/apple-touch-icon.png"/><link rel="icon" type="image/png" sizes="32x32" href="/img/favicon-32x32.png"/><link rel="icon" type="image/png" sizes="16x16" href="/img/favicon-16x16.png"/><link rel="mask-icon" href="/safari-pinned-tab.svg" color="#6fcd13"/><link rel="icon" href="/favicon.ico"/><meta name="format-detection" content="telephone=no"/><meta name="apple-mobile-web-app-title" content="nb-bet"/><meta name="application-name" content="nb-bet"/><meta name="msapplication-TileColor" content="#6fcd13"/><meta name="theme-color" content="#FAFAFA"/><link rel="manifest" href="/manifest.json" crossorigin="use-credentials"/><link rel="dns-prefetch" href="https://app.nb-bet.com"/><script type="application/ld+json">{"@context":"ht

In [4]:
soup = BeautifulSoup(response.text, 'lxml')

In [5]:
# Список всех команнд в Html
commands = soup.find_all('div', class_ = 'sc-94c6152a-8 jFyEfM')

In [6]:
# Названия комманд
command_A = commands[0].text
command_B = commands[1].text
print(command_A)
print(command_B)

Спартак Москва
Трактор


In [7]:
# Вывод счёта и сохранение в переменых
score = soup.find('div', class_ = 'sc-94c6152a-11 kTKQBv').text
game_score_A = float(score[0])
game_score_B = float(score[2])
print(game_score_A)
print(game_score_B)

2.0
2.0


In [8]:
# Ищем ставку на победу ничью поражение
bet = soup.find_all('div', class_ = 'sc-4b1ccd93-1 clOvIO') # это массив из трёх элементов

bet_win = float(bet[0].text[1:])
bet_draw = float(bet[1].text[1:])
bet_loss = float(bet[2].text[1:])

print(f"{bet_win}\n{bet_draw}\n{bet_loss}")

2.155
3.56
3.31


In [9]:
data_play = soup.find('div', class_ = 'sc-94c6152a-4 cFJoSp')
date_parts = data_play.text.split(' ')
Data = date_parts[0]
Data

'23.09.23'

# Сканируем данные первого периода

In [10]:

html_code = """
<div class="sc-72cccd94-0 beUBjZ wide"><h4 class="sc-3407ef55-0 hoRbFd">Статистика</h4><div><div class="sc-14e51187-2 eiGqUN default"><div class="sc-14e51187-0 bPcnZx"><div>Матч</div><div class="active">1-й период</div><div>2-й период</div><div>3-й период</div><div>Овертайм</div></div></div><div class="sc-14e51187-1 ehdVCB default"><div class="sc-72cccd94-1 dFAYkv"><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">7</div><div class="sc-72cccd94-6 fzdThe">Броски в створ</div><div class="sc-72cccd94-5 euOopX">13</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 35%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 65%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">0</div><div class="sc-72cccd94-6 fzdThe">Реализованные броски (%)</div><div class="sc-72cccd94-5 euOopX">7.69</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 0%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 100%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">2</div><div class="sc-72cccd94-6 fzdThe">Блокировано бросков</div><div class="sc-72cccd94-5 euOopX">3</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 40%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 60%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">12</div><div class="sc-72cccd94-6 fzdThe">Отраженные броски</div><div class="sc-72cccd94-5 euOopX">7</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 63%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div><div class="sc-72cccd94-9 jrhDzm" style="width: 37%;">&nbsp;</div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">92.31</div><div class="sc-72cccd94-6 fzdThe">Отраженные броски (%)</div><div class="sc-72cccd94-5 euOopX">100</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 48%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 52%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">1</div><div class="sc-72cccd94-6 fzdThe">Удаления</div><div class="sc-72cccd94-5 euOopX">1</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 50%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 50%;">&nbsp;</div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">2</div><div class="sc-72cccd94-6 fzdThe">Штрафное время</div><div class="sc-72cccd94-5 euOopX">2</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 50%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 50%;">&nbsp;</div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">3</div><div class="sc-72cccd94-6 fzdThe">Силовые приемы</div><div class="sc-72cccd94-5 euOopX">9</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 25%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 75%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">8</div><div class="sc-72cccd94-6 fzdThe">Выигранные вбрасывания</div><div class="sc-72cccd94-5 euOopX">9</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 47%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 53%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">47.06</div><div class="sc-72cccd94-6 fzdThe">Вбрасывания (%)</div><div class="sc-72cccd94-5 euOopX">52.94</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 47%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 53%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div></div></div></div></div></div></div>
"""

# Создаем объект BeautifulSoup
soup = BeautifulSoup(html_code, 'html.parser')

# Извлекаем все текстовые данные из элементов с классом "fzdThe" это заглавие Title
title_1_period = [element.text.strip() for element in soup.find_all(class_='fzdThe')]
value_command_A_1_period = [element.text.strip() for element in soup.find_all(class_='eSsGJj')]
value_command_B_1_period = [element.text.strip() for element in soup.find_all(class_='euOopX')]
print(title_1_period)
print(value_command_A_1_period)
print(value_command_B_1_period)


['Броски в створ', 'Реализованные броски (%)', 'Блокировано бросков', 'Отраженные броски', 'Отраженные броски (%)', 'Удаления', 'Штрафное время', 'Силовые приемы', 'Выигранные вбрасывания', 'Вбрасывания (%)']
['7', '0', '2', '12', '92.31', '1', '2', '3', '8', '47.06']
['13', '7.69', '3', '7', '100', '1', '2', '9', '9', '52.94']


In [ ]:
# @title Создание таблицы с результатами 1-го периода на основе стандартных заголовков

# Исходные данные
scanned_title = title_1_period

# Создаем DataFrame для исходной таблицы
df_original = pd.DataFrame(columns=standart_title)

# Заполняем значениями из отсканированных данных
for title, value in zip(scanned_title, value_command_A_1_period):
    df_original[title] = [value]

# Заполняем нулями для отсутствующих заголовков
for title in standart_title:
    if title not in scanned_title:
        df_original[title] = [0]

# Выводим получившийся DataFrame
df_original


,Броски в створ,Реализованные броски (%),Блокировано бросков,Отраженные броски,Отраженные броски (%),Удаления,Штрафное время,Силовые приемы,Выигранные вбрасывания,Вбрасывания (%),Шайбы в большинстве
0,7,0,2,12,92.31,1,2,3,8,47.06,0


# Сканируем данные второго периода

In [11]:

html_code = """
<div class="sc-72cccd94-0 beUBjZ wide"><h4 class="sc-3407ef55-0 hoRbFd">Статистика</h4><div><div class="sc-14e51187-2 eiGqUN default"><div class="sc-14e51187-0 bPcnZx"><div>Матч</div><div>1-й период</div><div class="active">2-й период</div><div>3-й период</div><div>Овертайм</div></div></div><div class="sc-14e51187-1 ehdVCB default"><div class="sc-72cccd94-1 dFAYkv"><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">12</div><div class="sc-72cccd94-6 fzdThe">Броски в створ</div><div class="sc-72cccd94-5 euOopX">12</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 50%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 50%;">&nbsp;</div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">8.33</div><div class="sc-72cccd94-6 fzdThe">Реализованные броски (%)</div><div class="sc-72cccd94-5 euOopX">0</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 100%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div><div class="sc-72cccd94-9 jrhDzm" style="width: 0%;">&nbsp;</div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">1</div><div class="sc-72cccd94-6 fzdThe">Блокировано бросков</div><div class="sc-72cccd94-5 euOopX">3</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 25%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 75%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">12</div><div class="sc-72cccd94-6 fzdThe">Отраженные броски</div><div class="sc-72cccd94-5 euOopX">11</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 52%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div><div class="sc-72cccd94-9 jrhDzm" style="width: 48%;">&nbsp;</div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">100</div><div class="sc-72cccd94-6 fzdThe">Отраженные броски (%)</div><div class="sc-72cccd94-5 euOopX">91.67</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 52%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div><div class="sc-72cccd94-9 jrhDzm" style="width: 48%;">&nbsp;</div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">1</div><div class="sc-72cccd94-6 fzdThe">Шайбы в большинстве</div><div class="sc-72cccd94-5 euOopX">0</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 100%;">&nbsp;<div class="sc-72cccd94-10 bNboJE"></div></div><div class="sc-72cccd94-9 jrhDzm" style="width: 0%;">&nbsp;</div></div></div><div class="sc-72cccd94-2 jTSINp"><div class="sc-72cccd94-3 eLUyLL"><div class="sc-72cccd94-4 eSsGJj">2</div><div class="sc-72cccd94-6 fzdThe">Силовые приемы</div><div class="sc-72cccd94-5 euOopX">2</div></div><div class="sc-72cccd94-7 fevuaJ"><div class="sc-72cccd94-8 gfJAfg" style="width: 50%;">&nbsp;</div><div class="sc-72cccd94-9 jrhDzm" style="width: 50%;">&nbsp;</div></div></div></div></div></div></div>
"""

# Создаем объект BeautifulSoup
soup = BeautifulSoup(html_code, 'html.parser')

# Извлекаем все текстовые данные из элементов с классом "fzdThe" это заглавие Title
title_2_period = [element.text.strip() for element in soup.find_all(class_='fzdThe')]
value_command_A_2_period = [element.text.strip() for element in soup.find_all(class_='eSsGJj')]
value_command_B_2_period = [element.text.strip() for element in soup.find_all(class_='euOopX')]
print(title_2_period)
print(value_command_A_2_period)
print(value_command_B_2_period)


['Броски в створ', 'Реализованные броски (%)', 'Блокировано бросков', 'Отраженные броски', 'Отраженные броски (%)', 'Шайбы в большинстве', 'Силовые приемы']
['12', '8.33', '1', '12', '100', '1', '2']
['12', '0', '3', '11', '91.67', '0', '2']


In [ ]:
# @title Создание таблицы с результатами 2-го периода на основе стандартных заголовков
# Исходные данные
scanned_title = title_2_period

# Создаем DataFrame для исходной таблицы
df_original = pd.DataFrame(columns=standart_title)

# Заполняем значениями из отсканированных данных
for title, value in zip(scanned_title, value_command_A_2_period):
    df_original[title] = [value]

# Заполняем нулями для отсутствующих заголовков
for title in standart_title:
    if title not in scanned_title:
        df_original[title] = [0]

# Выводим получившийся DataFrame
df_original


,A,B,D a t a,g a m e s c o r e A,g a m e s c o r e B,p e r i o d 1 s c o r e A,p e r i o d 1 s c o r e B,p e r i o d 2 s c o r e A,p e r i o d 2 s c o r e B,Команда A Броски в створ,...,Команда B Вбрасывания (%),Команда B Шайбы в большинстве,Команда B Шайбы в меньшинстве,Броски в створ,Реализованные броски (%),Блокировано бросков,Отраженные броски,Отраженные броски (%),Шайбы в большинстве,Силовые приемы
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,12,8.33,1,12,100,1,2


# Запись спарсиных данных

In [12]:
period_1_score_A = 0.0
period_1_score_B = 1.0
period_2_score_A = 1.0
period_2_score_B = 1.0

In [20]:
# @title Создание таблицы 0

headers_0 = ['A', 'B', 'Data', 'bet_win', 'bet_draw', 'bet_loss','game score A', 'game score B', 'period 1 score A', 'period 1 score B', 'period 2 score A', 'period 2 score B']
data_0 = [command_A, command_B, Data, bet_win, bet_draw, bet_loss, game_score_A, game_score_B, period_1_score_A, period_1_score_B, period_2_score_A, period_2_score_B]
df_0 = pd.DataFrame([data_0], columns=headers_0)
df_0

,A,B,Data,bet_win,bet_draw,bet_loss,game score A,game score B,period 1 score A,period 1 score B,period 2 score A,period 2 score B
0,Спартак Москва,Трактор,23.09.23,2.155,3.56,3.31,2.0,2.0,0.0,1.0,1.0,1.0


In [14]:
# @title Создание таблицы 1 периода

# Метки для команд A и B
team_labels = ['Команда A', 'Команда B']

# Создаем мультииндекс из меток и заголовков
multi_index = pd.MultiIndex.from_product([team_labels, title_1_period], names=['Команда', 'Показатель'])

# Создаем DataFrame с мультииндексом
df_1 = pd.DataFrame([value_command_A_1_period + value_command_B_1_period], columns=multi_index)

# Выводим DataFrame
df_1


Команда         Команда A                                               \
Показатель Броски в створ Реализованные броски (%) Блокировано бросков   
0                       7                        0                   2   

Команда                                                                     \
Показатель Отраженные броски Отраженные броски (%) Удаления Штрафное время   
0                         12                 92.31        1              2   

Команда                                                           \
Показатель Силовые приемы Выигранные вбрасывания Вбрасывания (%)   
0                       3                      8           47.06   

Команда         Команда B                                               \
Показатель Броски в створ Реализованные броски (%) Блокировано бросков   
0                      13                     7.69                   3   

Команда                                                                     \
Показатель Отраженные броски Отраженные броски (%) Удаления Штрафное время   
0                          7                   100        1              2   

Команда                                                           
Показатель Силовые приемы Выигранные вбрасывания Вбрасывания (%)  
0                       9                      9           52.94

In [21]:
# @title Объедеинение таблицы 0 и 1 периода
result_df = pd.concat([df_0, df_1], axis=1)
result_df

,A,B,Data,bet_win,bet_draw,bet_loss,game score A,game score B,period 1 score A,period 1 score B,...,"(Команда B, Броски в створ)","(Команда B, Реализованные броски (%))","(Команда B, Блокировано бросков)","(Команда B, Отраженные броски)","(Команда B, Отраженные броски (%))","(Команда B, Удаления)","(Команда B, Штрафное время)","(Команда B, Силовые приемы)","(Команда B, Выигранные вбрасывания)","(Команда B, Вбрасывания (%))"
0,Спартак Москва,Трактор,23.09.23,2.155,3.56,3.31,2.0,2.0,0.0,1.0,...,13,7.69,3,7,100,1,2,9,9,52.94


In [22]:
# @title Создание таблицы 2 периода

team_labels = ['Команда A', 'Команда B']
multi_index = pd.MultiIndex.from_product([team_labels, title_2_period], names=['Команда', 'Показатель'])
# Создаем DataFrame с мультииндексом
df_2 = pd.DataFrame([value_command_A_2_period + value_command_B_2_period], columns=multi_index)

# Выводим DataFrame
df_2

Команда         Команда A                                               \
Показатель Броски в створ Реализованные броски (%) Блокировано бросков   
0                      12                     8.33                   1   

Команда                                                                 \
Показатель Отраженные броски Отраженные броски (%) Шайбы в большинстве   
0                         12                   100                   1   

Команда                        Команда B                           \
Показатель Силовые приемы Броски в створ Реализованные броски (%)   
0                       2             12                        0   

Команда                                                                 \
Показатель Блокировано бросков Отраженные броски Отраженные броски (%)   
0                            3                11                 91.67   

Команда                                        
Показатель Шайбы в большинстве Силовые приемы  
0                            0              2

In [23]:
# @title Объедеинение таблицы 0 1 и 2 периода
result_df = pd.concat([result_df, df_2], axis=1)
result_df

,A,B,Data,bet_win,bet_draw,bet_loss,game score A,game score B,period 1 score A,period 1 score B,...,"(Команда A, Отраженные броски (%))","(Команда A, Шайбы в большинстве)","(Команда A, Силовые приемы)","(Команда B, Броски в створ)","(Команда B, Реализованные броски (%))","(Команда B, Блокировано бросков)","(Команда B, Отраженные броски)","(Команда B, Отраженные броски (%))","(Команда B, Шайбы в большинстве)","(Команда B, Силовые приемы)"
0,Спартак Москва,Трактор,23.09.23,2.155,3.56,3.31,2.0,2.0,0.0,1.0,...,100,1,2,12,0,3,11,91.67,0,2


In [24]:
# @title Стандартный заголовок таблицы
standart_title = ['A', 'B', 'D a t a', 'bet_win', 'bet_draw', 'bet_loss', 'g a m e   s c o r e   A', 'g a m e   s c o r e   B', 'p e r i o d   1   s c o r e   A',
 'p e r i o d   1   s c o r e   B', 'p e r i o d   2   s c o r e   A', 'p e r i o d   2   s c o r e   B', 'Команда A Броски в створ', 'Команда A Реализованные броски (%)',
 'Команда A Блокировано бросков', 'Команда A Отраженные броски', 'Команда A Отраженные броски (%)', 'Команда A Удаления', 'Команда A Штрафное время', 'Команда A Силовые приемы',
 'Команда A Выигранные вбрасывания', 'Команда A Вбрасывания (%)', 'Команда A Шайбы в большинстве', 'Команда A Шайбы в меньшинстве',
 'Команда B Броски в створ', 'Команда B Реализованные броски (%)', 'Команда B Блокировано бросков', 'Команда B Отраженные броски', 'Команда B Отраженные броски (%)',
 'Команда B Удаления', 'Команда B Штрафное время', 'Команда B Силовые приемы', 'Команда B Выигранные вбрасывания', 'Команда B Вбрасывания (%)',
 'Команда B Шайбы в большинстве', 'Команда B Шайбы в меньшинстве', 'Команда A Броски в створ', 'Команда A Реализованные броски (%)', 'Команда A Блокировано бросков',
 'Команда A Отраженные броски', 'Команда A Отраженные броски (%)', 'Команда A Удаления', 'Команда A Штрафное время', 'Команда A Силовые приемы',
 'Команда A Выигранные вбрасывания', 'Команда A Вбрасывания (%)', 'Команда A Шайбы в большинстве', 'Команда A Шайбы в меньшинстве',
 'Команда B Броски в створ', 'Команда B Реализованные броски (%)', 'Команда B Блокировано бросков', 'Команда B Отраженные броски', 'Команда B Отраженные броски (%)',
 'Команда B Удаления', 'Команда B Штрафное время', 'Команда B Силовые приемы', 'Команда B Выигранные вбрасывания', 'Команда B Вбрасывания (%)',
 'Команда B Шайбы в большинстве', 'Команда B Шайбы в меньшинстве']


In [25]:
result_df.columns

Index([                                      'A',
                                             'B',
                                          'Data',
                                       'bet_win',
                                      'bet_draw',
                                      'bet_loss',
                                  'game score A',
                                  'game score B',
                              'period 1 score A',
                              'period 1 score B',
                              'period 2 score A',
                              'period 2 score B',
                 ('Команда A', 'Броски в створ'),
       ('Команда A', 'Реализованные броски (%)'),
            ('Команда A', 'Блокировано бросков'),
              ('Команда A', 'Отраженные броски'),
          ('Команда A', 'Отраженные броски (%)'),
                       ('Команда A', 'Удаления'),
                 ('Команда A', 'Штрафное время'),
                 ('Команда A', 'Силовые приемы'),


In [26]:
result_df.values

array([['Спартак Москва', 'Трактор', '23.09.23', 2.155, 3.56, 3.31, 2.0,
        2.0, 0.0, 1.0, 1.0, 1.0, '7', '0', '2', '12', '92.31', '1', '2',
        '3', '8', '47.06', '13', '7.69', '3', '7', '100', '1', '2', '9',
        '9', '52.94', '12', '8.33', '1', '12', '100', '1', '2', '12',
        '0', '3', '11', '91.67', '0', '2']], dtype=object)

In [29]:

# Замените result_df на ваш фактический DataFrame с данными
result_df = pd.DataFrame(result_df)

# Создание стандартного DataFrame с названиями столбцов
standart_result_df = pd.DataFrame(columns=standart_title)

# Перенос данных из result_df в standart_result_df
for column in standart_result_df.columns:
    if column in result_df.columns:
        standart_result_df[column] = result_df[column]
    else:
        standart_result_df[column] = 0

# Вывод результата
print(standart_result_df)


                A        B      Data  game score A  game score B  \
0  Спартак Москва  Трактор  23.09.23           2.0           2.0   

   period 1 score A  period 1 score B  period 2 score A  period 2 score B  \
0               0.0               1.0               1.0               1.0   

   Команда A Броски в створ  ...  Команда B Блокировано бросков  \
0                         0  ...                              0   

   Команда B Отраженные броски  Команда B Отраженные броски (%)  \
0                            0                                0   

   Команда B Удаления  Команда B Штрафное время  Команда B Силовые приемы  \
0                   0                         0                         0   

   Команда B Выигранные вбрасывания  Команда B Вбрасывания (%)  \
0                                 0                          0   

   Команда B Шайбы в большинстве  Команда B Шайбы в меньшинстве  
0                              0                              0  

[1 rows x 57 colu

In [31]:
standart_result_df.shape

(1, 57)

In [32]:
standart_result_df

,A,B,Data,game score A,game score B,period 1 score A,period 1 score B,period 2 score A,period 2 score B,Команда A Броски в створ,...,Команда B Блокировано бросков,Команда B Отраженные броски,Команда B Отраженные броски (%),Команда B Удаления,Команда B Штрафное время,Команда B Силовые приемы,Команда B Выигранные вбрасывания,Команда B Вбрасывания (%),Команда B Шайбы в большинстве,Команда B Шайбы в меньшинстве
0,Спартак Москва,Трактор,23.09.23,2.0,2.0,0.0,1.0,1.0,1.0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
# @title Запись в файл
import pandas as pd
from openpyxl import load_workbook

# Открываем существующий файл Excel
file_path = 'output1.xlsx'
try:
    # Пытаемся получить существующий лист
    existing_df = pd.read_excel(file_path, sheet_name='Sheet1')
    # Сбрасываем индекс существующего DataFrame
    existing_df.reset_index(drop=True, inplace=True)

    # Добавляем новые данные к существующим данным
    combined_df = existing_df.append(standart_result_df, ignore_index=True)

    # Открываем файл Excel для записи
    with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        # Записываем объединенные данные
        combined_df.to_excel(writer, sheet_name='Sheet1', index=False)

except FileNotFoundError:
    # Если файл не существует, просто записываем новые данные
    standart_result_df.to_excel(file_path, sheet_name='Sheet1', index=False)


<ipython-input-35-7b014e80c6e2>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = existing_df.append(standart_result_df, ignore_index=True)


InvalidIndexError: ignored